In [2]:
import tensorflow as tf
from gensim.models import Doc2Vec
import numpy as np
from tensorflow.contrib import rnn 


C:\Users\88015\Anaconda3\envs\thesis\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
import logging
import os.path
import sys
import _pickle as pickle

program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))

2018-07-24 16:04:42,522 : INFO : running C:\Users\88015\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py -f C:\Users\88015\AppData\Roaming\jupyter\runtime\kernel-053937b1-763b-44a2-aca0-643ff5068728.json


In [4]:
model = Doc2Vec.load('./pretrained.d2v')

2018-07-24 16:04:42,657 : INFO : loading Doc2Vec object from ./pretrained.d2v
2018-07-24 16:04:43,969 : INFO : loading vocabulary recursively from ./pretrained.d2v.vocabulary.* with mmap=None
2018-07-24 16:04:43,970 : INFO : loading trainables recursively from ./pretrained.d2v.trainables.* with mmap=None
2018-07-24 16:04:43,970 : INFO : loading wv recursively from ./pretrained.d2v.wv.* with mmap=None
2018-07-24 16:04:43,971 : INFO : loading docvecs recursively from ./pretrained.d2v.docvecs.* with mmap=None
2018-07-24 16:04:43,972 : INFO : loaded ./pretrained.d2v


In [5]:
train_arrays = np.zeros((8000, 100))
train_labels = np.zeros((8000,2))

for i in range(4000):
    train_neg = "TRAIN_NEG_"+str(i)
    train_pos= "TRAIN_POS_"+str(i)
    prefix_train_pos = model.docvecs[train_pos] 
    prefix_train_neg = model.docvecs[train_neg] 
    train_arrays[i] = prefix_train_pos
    train_arrays[4000 + i] = prefix_train_neg
    train_labels[i][0] = 1
    train_labels[4000 + i][1] = 0

In [6]:
test_arrays = np.zeros((400, 100))
test_labels = np.zeros((400,2))

for i in range(200):
    test_neg = "TEST_NEG_"+str(i)
    test_pos= "TEST_POS_"+str(i)
    prefix_test_pos = model.docvecs[test_pos] 
    prefix_test_neg = model.docvecs[test_neg] 
    test_arrays[i] = prefix_test_pos
    test_arrays[200 + i] = prefix_test_neg
    test_labels[i][0] = 1
    test_labels[200 + i][1] = 0

In [16]:
#defining the model

#3 hidden layers
#nodes for each hidden layer

epochs = 6
n_classes = 2
batch_size = 128
chunk_size = 5
n_chunk = 20
rnn_size = 128

#height x width
x = tf.placeholder('float', [None, n_chunk, chunk_size])
y = tf.placeholder('float')

tf.reset_default_graph()

def recurrent_neural_network(data):
 
    
    
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size, n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunk, 0)
    
    lstm_cell= rnn_cell.BasicLSTMCell(rnn_size)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype = tf.float32) 
    

    
    output = tf.matmul(outputs[-1], layer['weights']) + layer['biases']
    
    
    return output

In [17]:
sess = tf.InteractiveSession()

In [19]:
def train_nural_network(x):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        _sentinel=None,
        labels= y,
        logits= prediction,
        dim=-1))
    #learning_rate = 0.001 default
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    global sess
    if(True):
#     with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        
        for epoch in range(epochs):
            epoch_loss = 0
            k = 0
            
            while k< len(train_arrays):
                start = k
                end = k+ batch_size
                batch_x = np.array(train_arrays[start:end])
                batch_y = np.array(train_labels[start:end])
                batch_x = batch_x.reshape((batch_size, n_chunk, chunk_size))
                
                i, c = sess.run([optimizer, cost], feed_dict = {x:batch_x, y:batch_y})
                epoch_loss += c
                k += batch_size
                
            #print('Epoch ', epoch, 'completed out of ', epochs, 'loss', epoch_loss, 'acc:',i )
            
        
        correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x:test_arrays.reshape((-1, n_chunk, chunk_size)),
                                          y: test_labels})*100, '%')
        
                
            
        
#train_nural_network(x)   

In [20]:
train_nural_network(x) 

UnboundLocalError: local variable 'x' referenced before assignment